# Lab 11: Trees

## <font color=DarkRed>Your Exercise: Parsing Expressions with Boolean Operators</font>

Modify the `build_parse_tree`, and `evaluate` functions to handle boolean operators (`and`, `or`, and `not`). Remember that `not` is a unary operator, so this will complicate your code somewhat.

*Note: While this was a workshop problem partly worked on in class, I have decided that it's challenging enough to be a homework question.*

## <font color=green>Your Solution</font>

*Use a variety of code, Markdown (text) cells below to create your solution. Nice outputs would be timing results, and even plots. You will be graded not only on correctness, but the clarity of your code, descriptive text and other output. Keep it succinct!*

In [19]:
# Import library
import operator

In [2]:
class Stack:
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return self.items == []
    
    def push(self,item):
        self.items.append(item)
    
    def pop(self):
        if self.isEmpty():
            raise IndexError("Pop from an empty stack")
        item = self.items[-1]
        del self.items[-1]
        return item
    
    def peek(self):
        return self.items[-1]
    
    def size(self):
        return len(self.items)
    
    def __repr__(self):
        return "Stack: {}".format(self.items)
    
    def __str__(self):
        return "Your Stack: {}".format(self.items)

In [3]:
class BinaryTree:
    def __init__(self, rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None
    
    def insertLeft(self, newNode):
        if self.leftChild == None:
            self.leftChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.leftChild = self.leftChild
            self.leftChild = t
    
    def insertRight(self, newNode):
        if self.rightChild == None:
            self.rightChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.rightChild = self.rightChild
            self.rightChild = t
    
    def getRightChild(self):
        return self.rightChild
    
    def getLeftChild(self):
        return self.leftChild
    
    def setRootVal(self, obj):
        self.key = obj
    
    def getRootVal(self):
        return self.key
    
    def __repr__(self):
        return "BinaryTree: [root: {}, leftChild: {}, rightChild: {}]\n".format(self.key, self.leftChild, self.rightChild)

In [4]:
def buildParseTree(fpexp):
    
    # Handles expression that does not have spaces between every characters
    
    # Solution: Just add extra spaces before and after any mathematical expressions.
    #           split() will take care of redundant spaces
    
    # Parse the input expression string
    newExp = ""
    for char in fpexp:
        if char in ['(','+', '-', '*', '/', ')']:
            newExp += ' ' + char + ' '
        else:
            newExp += char
    # Remove redundant leading and trailing spaces
    newExp.strip()
    # Split the expression into list by white space
    fplist = newExp.split()
    # Create an empty stack for parsing
    pStack = Stack()
    # Initialize an empty binary tree
    eTree = BinaryTree('')
    # Push the tree onto the stack
    pStack.push(eTree)
    # Set current tree
    currentTree = eTree
    # For each element in the parsed list
    for i in fplist:
        # If the current element is left open parenthesis
        if i == '(':
            # Insert empty node to the left of tree
            currentTree.insertLeft('')
            # Push the current tree onto the stack
            pStack.push(currentTree)
            # Reset the current tree to the left child (because the old current tree has been pushed downward after the insertion)
            currentTree = currentTree.getLeftChild()
        # If the element is not any operator
        elif i not in ['+', '-', '*', '/', ')','and','or','not']:
            # Set root value
            currentTree.setRootVal(int(i))
            # Pop a tree out of the stack
            parent = pStack.pop()
            # Reset the current tree
            currentTree = parent
        elif i == 'not':
            # Set root value
            currentTree.setRootVal(i)
            # Insert empty tree to right child
            currentTree.insertRight('')
            # Push current tree onto stack
            pStack.push(currentTree)
            # Set current tree to right child
            currentTree = currentTree.getRightChild()
        elif i in ['+', '-', '*', '/','and','or']:
            # Set root value
            currentTree.setRootVal(i)
            # Insert empty tree to right child
            currentTree.insertRight('')
            # Push current tree onto Stack
            pStack.push(currentTree)
            # Set current tree to right child
            currentTree = currentTree.getRightChild()
        elif i == ')':
            # Pop item out of stack. Set that to current tree
            currentTree = pStack.pop()
        else:
            raise ValueError('{} is not implemented'.format(i))
    return eTree

In [5]:
def evaluate(parseTree):
    opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv, 
             'and':operator.add, 'or':'or', 'not':operator.ne}

    leftC = parseTree.getLeftChild()
    rightC = parseTree.getRightChild()
    
    # Case for "not" as negation, not inequality test
    if leftC is None and rightC:
        fn = operator.not_
        return fn(evaluate(rightC))
    elif leftC and rightC:
        fn = opers[parseTree.getRootVal()]
        # Special case for 'or'
        if fn == 'or':
            alist = []
            alist.append(evaluate(leftC))
            alist.append(evaluate(rightC))
            return alist
        else:
            return fn(evaluate(leftC),evaluate(rightC))
    else:
        return parseTree.getRootVal()

## Testing

Test `build_parse_tree`, and `evaluate` to show that boolean expressions (with, or without arithmetic expressions mixed), work as expected.

In [6]:
pt = buildParseTree("( (10+5) * 3 )")
evaluate(pt)

45

In [7]:
pt = buildParseTree("( (1+5) * (3-1) )")
evaluate(pt)

12

In [8]:
pt = buildParseTree("( ( 2 and 2 ) * 5 )")
evaluate(pt)

20

In [9]:
pt = buildParseTree("( ( 4 - 26 ) and 5 )")
evaluate(pt)

-17

In [10]:
pt = buildParseTree("( ( 1 and 2 ) not 3 )")
evaluate(pt)

False

In [11]:
pt = buildParseTree("not ( ( 1 and 2 ) not 3 )")
evaluate(pt)

True

In [12]:
pt = buildParseTree("( ( 3 * 7 ) not 21 )")
evaluate(pt)

False

In [13]:
pt = buildParseTree("( 12 or 3 )")
evaluate(pt)

[12, 3]

In [14]:
pt = buildParseTree("( ( 1 and 6 ) or 3 )")
evaluate(pt)

[7, 3]

In [13]:
not False

True

In [14]:
not(True)

False

In [15]:
not(3)

False

In [16]:
not(10)

False

In [17]:
not(0)

True

In [18]:
not(-1)

False

In [20]:
operator.not_(0)

True

In [21]:
operator.not_(1)

False